# 测试DeepSeek v3的后置断言验证

使用DeepSeek替代Kimi，测试知识驱动的后置断言验证

In [ ]:
import os
import sys
from pathlib import Path
import shutil

# 添加项目路径
sys.path.insert(0, str(Path.cwd()))

from core.react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel

# 设置工作目录
work_dir = "/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/output/deepseek_test"

# 清空工作目录
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
os.makedirs(work_dir)

print(f"工作目录: {work_dir}")

## 配置DeepSeek v3

In [ ]:
# DeepSeek v3配置
deepseek_config = {
    "llm_model": "deepseek-chat",
    "llm_base_url": "https://api.deepseek.com/v1",
    "llm_api_key_env": "DEEPSEEK_API_KEY",
    "llm_temperature": 0
}

print(f"使用LLM: {deepseek_config['llm_model']}")

## 测试PSM生成的后置断言验证

In [ ]:
# 创建PSM生成Agent
config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    knowledge_files=["knowledge/mda/pim_to_psm_knowledge.md"],
    enable_project_exploration=False,
    **deepseek_config
)

psm_agent = GenericReactAgent(config, name="psm_generation_agent")
psm_agent.interface = """PSM生成专家

# 重要提醒
根据knowledge文件，生成PSM必须包含5个核心章节
"""

# 执行任务
result = psm_agent.execute_task("""
# PIM文件
/home/guci/aiProjects/mda/pim-compiler/examples/blog.md

# 任务
根据上面的PIM生成PSM，文件名是blog_psm.md

# 成功判定条件
1. blog_psm.md文件存在
2. 包含Domain Models章节
3. 包含Service Layer章节
4. 包含REST API Design章节
5. 包含Application Configuration章节
6. 包含Testing Specifications章节
""")

print(f"\n执行结果: {result}")

## 验证文件是否真的生成了

In [ ]:
# 检查PSM文件
psm_file = os.path.join(work_dir, "blog_psm.md")

print("="*60)
print("验证结果")
print("="*60)

if os.path.exists(psm_file):
    print(f"✅ PSM文件存在: {psm_file}")
    with open(psm_file, 'r', encoding='utf-8') as f:
        content = f.read()
        sections = [
            "Domain Models",
            "Service Layer",
            "REST API Design",
            "Application Configuration",
            "Testing Specifications"
        ]
        for section in sections:
            if section in content:
                print(f"  ✅ 包含章节: {section}")
            else:
                print(f"  ❌ 缺少章节: {section}")
else:
    print(f"❌ PSM文件不存在")
    print("\n问题：Agent没有生成文件却可能报告成功")

## 测试代码生成的后置断言验证

In [ ]:
# 创建代码生成Agent
config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    knowledge_files=["knowledge/mda/generation_knowledge.md"],
    enable_project_exploration=False,
    **deepseek_config
)

generation_agent = GenericReactAgent(config, name="generation_agent")
generation_agent.interface = """代码生成专家

# 重要原则（来自generation_knowledge.md）
0. 必须验证成功条件
1. 专注生成，不做调试
2. 增量生成
3. 必须生成单元测试
"""

# 执行任务
result = generation_agent.execute_task("""
# PSM文件
blog_psm.md

# 任务
根据上面的PSM生成代码

# 成功判定条件
1. 生成了完整的FastAPI应用代码（app/main.py存在）
2. 生成了完整的单元测试代码（tests/目录存在）
3. 生成了readme.md
""")

print(f"\n执行结果: {result}")

## 验证代码文件

In [ ]:
# 检查生成的文件
files_to_check = [
    "app/main.py",
    "tests",
    "readme.md"
]

print("="*60)
print("验证代码生成结果")
print("="*60)

for file in files_to_check:
    file_path = os.path.join(work_dir, file)
    if os.path.exists(file_path):
        if os.path.isdir(file_path):
            print(f"✅ 目录存在: {file}")
            # 列出目录内容
            for item in os.listdir(file_path):
                print(f"    - {item}")
        else:
            print(f"✅ 文件存在: {file}")
    else:
        print(f"❌ 不存在: {file}")

## 分析DeepSeek vs Kimi的表现

对比两个模型在后置断言验证方面的表现：

1. **是否真的生成了文件**
2. **是否验证了成功条件**
3. **失败时是否意识到并报告**